In [1]:
import math
import numpy as np
from scipy import integrate

### Montgomeries Transformation

In [2]:
def MTransformation(a,p):
    exp = math.ceil(math.log(p, 2))
    R = pow(2, exp)
    return (a*R)%p

In [3]:
print(MTransformation(2,11))

10


## MMM

In [4]:
def inv(a, p):
    for i in range(1, p):
        if (a*i)%p == 1:
            return i

In [5]:
def MMM(a,b,p):
    return MTransformation(a*b, p)

In [6]:
print(MMM(2,5,7))

3


## MMM reduction

In [7]:
def find_v(p):
    exp = math.ceil(math.log(p, 2))
    R = pow(2, exp)
    Rinv = inv(R, p)
    v = (R*Rinv-1)/p
    return (R, int(v))

In [8]:
def MMMr(a, b, p):
    D = MTransformation(a, p)*MTransformation(b, p)
    R, v = find_v(p)
    m = ((D%R)*v)%R
    U = (D+m*p)/R
    C = U
    if U >= p:
        C = U-p
    return int(C)

In [9]:
print(MMMr(6,14, 23))

20


##  Left-to-right Montgomery ladder with MM algorithm

In [10]:
def MM(a,b,p):
    exp = math.ceil(math.log(p, 2))
    Rinv = inv(pow(2, exp), p)
    return (a*b*Rinv)%p

In [11]:
def LtRMontLadder(m, kb, p):
    exp = math.ceil(math.log(p, 2))
    R = pow(2, exp)
    Rinv = inv(R, p)
    R0 = (m*R)%p
    R1 = MM(R0, R0, p)
    for i in range(len(kb)-1, 1, -1):
        if kb[i] == 0:
            R1 = MM(R0, R1, p)
            R0 = MM(R0,R0,p)
        else:
            R0 = MM(R0, R1, p)
            R1 = MM(R1,R1,p)
    return (R0*Rinv)%p

In [12]:
p = 11
kb = bin(21)
m = 3
print(LtRMontLadder(m, kb, p))

5


In [13]:
p = 7
kb = bin(17)
m = 3
print(LtRMontLadder(m, kb, p))

6


## Attack

In [21]:
def uM(ER):
    N = len(ER)
    l = len(ER[0])
    l0 = l//2
    pR_E = 0
    for i in range(N):
        for j in range(0, l, 2):
            pR_E += ER[i][j]
    return pR_E/(N*l0)

def uQ(ER):
    N = len(ER)
    l = len(ER[0])
    l0 = l//2
    pn_E = 0
    for i in range(N):
        for j in range(1, l, 2):
            pn_E += ER[i][j]
    return pn_E/(N*l0)

In [27]:
def p_Estimators(ER):
    um = uM(ER)
    uq = uQ(ER)
    pR_e = (12*(uq-um))/(1+6*uq-8*um)
    pn_e = 4*um-3*uq
    return (pR_e,pn_e)

In [32]:
p_Estimators([[1,0,1,1,0,1,1,1],[1,1,0,0,1,1,0,1],[1,1,0,1,1,1,1,1]])

(3.0, 0.16666666666666652)

In [73]:
def vErrors(u):
    r = []
    for i in range(pow(2,2*u)):
        bf = list(bin(i)[2:])
        temp = [0]*((2*u)-len(bf))+[eval(i) for i in bf]
        if temp not in r:
            r.append(temp)
    return r
vErrors(2)

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

In [2]:
def ham(V):
    c = 0
    for i in V:
        if i == 1:
            c += 1
    return c

In [6]:
def prob_u2(vt, ER, pR, pn):
    prob = 0
    if vt == [0, 0] or vt == [1,1]:
        if ER == [0,0,0,0]:
            return 1+(1/264)*(pR**8)-(1/72)*(pR**6)-(11/336)*(pR**5)+(17/504)*(pR**4)+(7/90)*(pR**3)+(1/3)*(pR**2)-(7/6)*(pR)
        elif ER == [0,0,0,1]:
            return -(1/264)*(pR**8)+(1/72)*(pR**6)+(1/84)*(pR**5)+(1/72)*(pR**4)-(7/90)*(pR**3)+(1/8)*(pR**2)+(1/4)*pR
        elif ER == [0,0,1,0]:
            return -(1/264)*(pR**8)+(1/72)*(pR**6)+(1/48)*(pR**5)-(17/504)*(pR**4)-(5/24)*(pR**2)+(1/3)*pR
        elif ER == [0,0,1,1]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/72)*(pR**4)+(1/8)*(pR**2)
        elif ER == [0,1,0,0]:
            return -(1/264)*(pR**8)+(11/336)*(pR**5)+(1/72)*(pR**4)-(1/36)*(pR**3)-(5/24)*(pR**2)+(1/4)*(pR)
        elif ER == [0,1,0,1]:
            return (1/264)*(pR**8)-(1/84)*(pR**5)-(1/72)*(pR**4)+(1/36)*(pR**3)
        elif ER == [0,1,1,0]:
            return (1/264)*(pR**8)-(1/48)*(pR**5)-(1/72)*(pR**4)+(1/12)*(pR**2)
        elif ER == [0,1,1,1]:
            return -(1/264)*(pR**8)+(1/72)*(pR**4)
        elif ER == [1,0,0,0]:
            return (1/7)*((1/12)*(pR**5)-(1/3)*(pR**4))-(1/264)*(pR**8)+(1/72)*(pR**6)+(1/48)*(pR**5)-(1/20)*(pR**3)-(1/8)*(pR**2)+(1/3)*pR
        elif ER == [1,0,0,1]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/84)*(pR**5)+(1/20)*(pR**3)
        elif ER == [1,0,1,0]:
            return (1/264)*(pR**8)-(1/72)*(pR**6)-(1/48)*(pR**5)+(1/21)*(pR**4)
        elif ER == [1,0,1,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,0,0]:
            return (1/264)*(pR**8)-(11/336)*(pR**5)+(1/8)*(pR**2)
        elif ER == [1,1,0,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,1,0]:
            return -(1/264)*(pR**8)+(1/84)*(pR**5)
        else:
            return (1/264)*(pR**8)
    elif vt == [0,1] or vt == [1,0]:
        if ER == [0,0,0,0]:
            return 1+(1/360)*(pR**8)-(1/200)*(pR**6)-(13/504)*(pR**5)+(1/240)*(pR**4)+(7/90)*(pR**3)+(13/36)*(pR**2)-(7/6)*(pR)
        elif ER == [0,0,0,1]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(1/72)*(pR**5)+(1/240)*(pR**4)-(17/72)*(pR**2)+(1/3)*pR
        elif ER == [0,0,1,0]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(1/84)*(pR**5)+(1/40)*(pR**4)-(7/90)*(pR**3)-(1/8)*(pR**2)+(1/4)*pR
        elif ER == [0,0,1,1]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/40)*(pR**4)+(1/8)*(pR**2)
        elif ER == [0,1,0,0]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)+(13/504)*(pR**5)-(1/48)*(pR**4)-(1/36)*(pR**3)-(1/8)*(pR**2)+(1/4)*(pR)
        elif ER == [0,1,0,1]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/72)*(pR**5)+(1/48)*(pR**4)
        elif ER == [0,1,1,0]:
            return (1/360)*(pR**8)-(1/200)*(pR**6)-(1/84)*(pR**5)+(1/36)*(pR**3)
        elif ER == [0,1,1,1]:
            return -(1/360)*(pR**8)+(1/200)*(pR**6)
        elif ER == [1,0,0,0]:
            return -(1/360)*(pR**8)+(13/504)*(pR**5)+(1/40)*(pR**4)-(1/20)*(pR**3)-(17/72)*(pR**2)+(1/3)*pR
        elif ER == [1,0,0,1]:
            return (1/360)*(pR**8)-(1/72)*(pR**5)-(1/40)*(pR**4)+(1/9)*(pR**2)
        elif ER == [1,0,1,0]:
            return (1/360)*(pR**8)-(1/84)*(pR**5)-(1/40)*(pR**4)+(1/20)*(pR**3)
        elif ER == [1,0,1,1]:
            return -(1/360)*(pR**8)+(1/40)*(pR**4)
        elif ER == [1,1,0,0]:
            return (1/360)*(pR**8)-(13/504)*(pR**5)+(1/8)*(pR**2)
        elif ER == [1,1,0,1]:
            return -(1/360)*(pR**8)+(1/72)*(pR**5)
        elif ER == [1,1,1,0]:
            return -(1/360)*(pR**8)+(1/84)*(pR**5)
        else:
            return (1/360)*(pR**8)

In [83]:
def Attack(ER):
    u = 2
    pR_E, pn_E = p_Estimators(ER)
    prob_l = []
    Err_v = vErrors(u)
    # i = u-1, 2(u-1), 3(u-1)...
    #while i < l/u
    for j in range(pow(2, u)):
        if bin(j)[-1] == '0':
            for v in Err_v:
                #prob

In [84]:
Attack([[1,0,1,1,0,1,1,1],[1,1,0,0,1,1,0,1],[1,1,0,1,1,1,1,1]])

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

In [8]:
z = [1,2,3,4]
z = z[::-1]
z

[4, 3, 2, 1]